### Retrieving Google businesses' status
       OPERATIONAL/CLOSED_PERMANENTLY/CLOSED_TEMPORARILY

In [4]:
import pandas as pd
import googlemaps
import time


API_KEY = "XXX"
gmaps = googlemaps.Client(key=API_KEY)

# Load business data
file_path = "../../Cleaned Data/fullcoffeeshops.csv"
business_data = pd.read_csv(file_path)

# ✅ Function to search for a business
def get_business_info(name, lat, lon):
    try:
        # Search for places using name and a small radius
        results = gmaps.places_nearby(
            location=(lat, lon), 
            radius=100,  # Search within 500 meters
            name=name
        )

        if results and "results" in results and results["results"]:
            place = results["results"][0]  # Get the first matching result
            
            # Extract business details
            place_id = place["place_id"]
            business_status = place.get("business_status", "UNKNOWN")  # Check if Open, Closed, etc.
            
            return place_id, business_status
        else:
            return None, "NOT FOUND"

    except Exception as e:
        print(f"Error fetching info for {name}: {e}")
        return None, "ERROR"

# ✅ Apply the function to all businesses
business_data["Google Place ID"] = None
business_data["Business Status"] = None

for index, row in business_data.iterrows():
    place_id, status = get_business_info(row["Name"], row["Latitude"], row["Longitude"])
    business_data.at[index, "Google Place ID"] = place_id
    business_data.at[index, "Business Status"] = status

    # Avoid hitting API rate limits
    time.sleep(1)  # Wait 1 second per request

# ✅ Save the updated data
output_file = "fullcoffeeshpos_with_google_status_NAME.csv"
business_data.to_csv(output_file, index=False)

print(f"✅ Updated business data saved to: {output_file}")


✅ Updated business data saved to: fullcoffeeshpos_with_google_status_NAME.csv


In [6]:
import pandas as pd

def count_records_by_categories(csv_file, column_name, category_values):
    """
    Reads a CSV file and counts the number of records where column_name matches any value in category_values.
    
    :param csv_file: Path to the CSV file
    :param column_name: The column to filter by
    :param category_values: A list of categories to count
    :return: Dictionary with category counts
    """
    try:
        df = pd.read_csv(csv_file)
        counts = {category: (df[column_name] == category).sum() for category in category_values}
        return counts
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
if __name__ == "__main__":
    csv_path = "fullcoffeeshpos_with_google_status_NAME.csv"  # Update with actual file path
    column = "Business Status"  # Update with actual column name
    categories = ["OPERATIONAL", "CLOSED_PERMANENTLY", "CLOSED_TEMPORARILY", "NOT FOUND"]  # Update with actual category values
    
    result = count_records_by_categories(csv_path, column, categories)
    if result is not None:
        for category, count in result.items():
            print(f"Number of records in category '{category}': {count}")


Number of records in category 'OPERATIONAL': 947
Number of records in category 'CLOSED_PERMANENTLY': 321
Number of records in category 'CLOSED_TEMPORARILY': 64
Number of records in category 'NOT FOUND': 1076


In [1]:
import pandas as pd
import googlemaps
import time


API_KEY = "XXX"
gmaps = googlemaps.Client(key=API_KEY)

# Load business data
file_path = "../../Cleaned Data/fullrestaurants.csv"
business_data = pd.read_csv(file_path)

# Function to search for a business
def get_business_info(name, lat, lon):
    try:
        # Search for places using name and a small radius
        results = gmaps.places_nearby(
            location=(lat, lon), 
            radius=100,  # Search within 500 meters
            keyword=name
        )

        if results and "results" in results and results["results"]:
            place = results["results"][0]  # Get the first matching result
            
            # Extract business details
            place_id = place["place_id"]
            business_status = place.get("business_status", "UNKNOWN")  # Check if Open, Closed, etc.
            
            return place_id, business_status
        else:
            return None, "NOT FOUND"

    except Exception as e:
        print(f"Error fetching info for {name}: {e}")
        return None, "ERROR"

# Apply the function to all businesses
business_data["Google Place ID"] = None
business_data["Business Status"] = None

for index, row in business_data.iterrows():
    place_id, status = get_business_info(row["Name"], row["Latitude"], row["Longitude"])
    business_data.at[index, "Google Place ID"] = place_id
    business_data.at[index, "Business Status"] = status

    # Avoid hitting API rate limits
    time.sleep(1)  # Wait 1 second per request

# Save the updated data
output_file = "fullrestaurants_with_google_status.csv"
business_data.to_csv(output_file, index=False)

print(f"Updated business data saved to: {output_file}")


Error fetching info for Eastern Broasted (بروستد الشرقية): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
✅ Updated business data saved to: fullrestaurants_with_google_status.csv


In [3]:
import pandas as pd

def count_records_by_categories(csv_file, column_name, category_values):
    """
    Reads a CSV file and counts the number of records where column_name matches any value in category_values.
    
    :param csv_file: Path to the CSV file
    :param column_name: The column to filter by
    :param category_values: A list of categories to count
    :return: Dictionary with category counts
    """
    try:
        df = pd.read_csv(csv_file)
        counts = {category: (df[column_name] == category).sum() for category in category_values}
        return counts
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
if __name__ == "__main__":
    csv_path = "fullrestaurants_with_google_status.csv"  # Update with actual file path
    column = "Business Status"  # Update with actual column name
    categories = ["OPERATIONAL", "CLOSED_PERMANENTLY", "CLOSED_TEMPORARILY", "NOT FOUND"]  # Update with actual category values
    
    result = count_records_by_categories(csv_path, column, categories)
    if result is not None:
        for category, count in result.items():
            print(f"Number of records in category '{category}': {count}")


Number of records in category 'OPERATIONAL': 1526
Number of records in category 'CLOSED_PERMANENTLY': 399
Number of records in category 'CLOSED_TEMPORARILY': 60
Number of records in category 'NOT FOUND': 1194


In [9]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import googlemaps
import time
from geopy.distance import geodesic

API_KEY = "XXX"
gmaps = googlemaps.Client(key=API_KEY)

# Load business data
file_path = "../../Cleaned Data/fullcoffeeshops.csv"
business_data = pd.read_csv(file_path)

# ✅ Function to search for a business and find the closest match
def get_business_info(name, lat, lon):
    try:
        # Search for places by name (without lat/lon constraint)
        results = gmaps.places(query=name)

        if results and "results" in results and results["results"]:
            # List of candidates with distances
            candidates = []

            for place in results["results"]:
                place_lat = place["geometry"]["location"]["lat"]
                place_lon = place["geometry"]["location"]["lng"]
                distance = geodesic((lat, lon), (place_lat, place_lon)).meters

                candidates.append((place, distance))

            # Sort by distance and get the closest one
            candidates.sort(key=lambda x: x[1])

            if candidates:
                closest_place, min_distance = candidates[0]

                # ❌ If the closest match is more than 200m away, return NOT FOUND
                if min_distance > 200:
                    return None, "NOT FOUND", min_distance

                place_id = closest_place["place_id"]
                business_status = closest_place.get("business_status", "UNKNOWN")

                return place_id, business_status, min_distance

        return None, "NOT FOUND", None

    except Exception as e:
        print(f"Error fetching info for {name}: {e}")
        return None, "ERROR", None

# ✅ Apply the function to all businesses
business_data["Google Place ID"] = None
business_data["Business Status"] = None
business_data["Distance (m)"] = None

for index, row in business_data.iterrows():
    place_id, status, distance = get_business_info(row["Name"], row["Latitude"], row["Longitude"])
    business_data.at[index, "Google Place ID"] = place_id
    business_data.at[index, "Business Status"] = status
    business_data.at[index, "Distance (m)"] = distance

    # Avoid hitting API rate limits
    time.sleep(1)  # Wait 1 second per request

# ✅ Save the updated data
output_file = "fullcoffeeshpos_with_google_status_UPDATED.csv"
business_data.to_csv(output_file, index=False)

print(f"✅ Updated business data saved to: {output_file}")


✅ Updated business data saved to: fullcoffeeshpos_with_google_status_UPDATED.csv


In [20]:
import pandas as pd

def count_records_by_categories(csv_file, column_name, category_values):
    """
    Reads a CSV file and counts the number of records where column_name matches any value in category_values.
    
    :param csv_file: Path to the CSV file
    :param column_name: The column to filter by
    :param category_values: A list of categories to count
    :return: Dictionary with category counts
    """
    try:
        df = pd.read_csv(csv_file)
        counts = {category: (df[column_name] == category).sum() for category in category_values}
        return counts
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
if __name__ == "__main__":
    csv_path = "fullcoffeeshpos_with_google_status_UPDATED.csv"  # Update with actual file path
    column = "Business Status"  # Update with actual column name
    categories = ["OPERATIONAL", "CLOSED_PERMANENTLY", "CLOSED_TEMPORARILY", "NOT FOUND"]  # Update with actual category values
    
    result = count_records_by_categories(csv_path, column, categories)
    if result is not None:
        for category, count in result.items():
            print(f"Number of records in category '{category}': {count}")


Number of records in category 'OPERATIONAL': 762
Number of records in category 'CLOSED_PERMANENTLY': 252
Number of records in category 'CLOSED_TEMPORARILY': 55
Number of records in category 'NOT FOUND': 1340


In [21]:
import pandas as pd
import googlemaps
import time
from geopy.distance import geodesic

API_KEY = "XXX"
gmaps = googlemaps.Client(key=API_KEY)

# Load business data
file_path = "../../Cleaned Data/fullrestaurants.csv"
business_data = pd.read_csv(file_path)

# ✅ Function to search for a business and find the closest match
def get_business_info(name, lat, lon):
    try:
        # Search for places by name (without lat/lon constraint)
        results = gmaps.places(query=name)

        if results and "results" in results and results["results"]:
            # List of candidates with distances
            candidates = []

            for place in results["results"]:
                place_lat = place["geometry"]["location"]["lat"]
                place_lon = place["geometry"]["location"]["lng"]
                distance = geodesic((lat, lon), (place_lat, place_lon)).meters

                candidates.append((place, distance))

            # Sort by distance and get the closest one
            candidates.sort(key=lambda x: x[1])

            if candidates:
                closest_place, min_distance = candidates[0]

                # ❌ If the closest match is more than 200m away, return NOT FOUND
                if min_distance > 200:
                    return None, "NOT FOUND", min_distance

                place_id = closest_place["place_id"]
                business_status = closest_place.get("business_status", "UNKNOWN")

                return place_id, business_status, min_distance

        return None, "NOT FOUND", None

    except Exception as e:
        print(f"Error fetching info for {name}: {e}")
        return None, "ERROR", None

# ✅ Apply the function to all businesses
business_data["Google Place ID"] = None
business_data["Business Status"] = None
business_data["Distance (m)"] = None

for index, row in business_data.iterrows():
    place_id, status, distance = get_business_info(row["Name"], row["Latitude"], row["Longitude"])
    business_data.at[index, "Google Place ID"] = place_id
    business_data.at[index, "Business Status"] = status
    business_data.at[index, "Distance (m)"] = distance

    # Avoid hitting API rate limits
    time.sleep(1)  # Wait 1 second per request

# ✅ Save the updated data
output_file = "fullrestaurants_with_google_status_UPDATED.csv"
business_data.to_csv(output_file, index=False)

print(f"✅ Updated business data saved to: {output_file}")


✅ Updated business data saved to: fullrestaurants_with_google_status_UPDATED.csv


In [ ]:
import pandas as pd

# Load CSV file
file_path = '../../Data Preprocessing/Foursquare/final_restaurants_dataset_cleaned.csv'  
df = pd.read_csv(file_path)

# Specify the column to check for duplicates
column_name = "Google Place ID"  # Replace with your actual column name

# Find duplicated values
duplicates = df[df.duplicated(subset=[column_name], keep=False)]

# Display results
if not duplicates.empty:
    print(f"⚠️ Found {duplicates.shape[0]} duplicated values in column '{column_name}':")
    print(duplicates)
else:
    print(f"✅ No duplicated values found in column '{column_name}'.")

# Optionally, save the duplicate rows to a separate CSV file
duplicates.to_csv("duplicated_values.csv", index=False)


✅ No duplicated values found in column 'Google Place ID'.


: 